In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from spektral.layers import GCNConv
from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

from postprocess.threshold import flood_threshold, drought_threshold, flood_threshold_t1, drought_threshold_t1
from preprocess.GraphTransformerPrerocess import graph_water_transformer_cov_process_1
from preprocess.graph import graph_topology_5_1

In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Parameters

In [18]:
# ====== preprocessing parameters ======
n_hours = 72
K = 24 
masked_value = 1e-10
split_1 = 0.8
split_2 = 0.9
sigma2 = 0.1
epsilon = 0.5

### Preprocess

In [19]:
train_cov, val_cov, test_cov, \
train_tws_reshape, val_tws_reshape, test_tws_reshape, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = graph_water_transformer_cov_process_1(n_hours, K, masked_value, split_1, split_2)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
      dtype='object')


In [20]:
print(train_cov.shape, val_cov.shape, test_cov.shape, 
train_tws_reshape.shape, val_tws_reshape.shape, test_tws_reshape.shape, \
train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(77069, 96, 12) (9634, 96, 12) (19268, 96, 12) (77069, 5, 72) (9634, 5, 72) (19268, 5, 72) (77069, 96) (9634, 96) (19268, 96)


### Graph topology

In [21]:
train_adj_mat, val_adj_mat, test_adj_mat = graph_topology_5_1(n_hours, K, sigma2, epsilon, len(train_ws_y), len(val_ws_y), len(test_ws_y))

node_indices: [0 0 0 0 1 1 2 2 3 3 4 4] 
neighbor_indices: [1 2 3 4 0 2 0 1 0 4 0 3]
number of nodes: 5, number of edges: 12


### Model

In [36]:
saved_model = load_model('saved_models/WaLeF_gtn_p.h5', custom_objects={'GCNConv': GCNConv})

yhat = saved_model.predict([test_cov, test_tws_reshape, test_adj_mat])

inv_yhat = ws_scaler.inverse_transform(yhat)
inv_y = ws_scaler.inverse_transform(test_ws_y)
print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)


# compute time steps and areas over and under thresholds
inv_yhat_reshape = inv_yhat.reshape((-1, 24, 4))
inv_y_reshape = inv_y.reshape((-1, 24, 4))
print("inv_y_reshape.shape, inv_yhat_reshape.shape", inv_y_reshape.shape, inv_yhat_reshape.shape)

603/603 [==============================] - 2s 3ms/step
inv_y.shape, inv_yhat.shape (19268, 96) (19268, 96)
inv_y_reshape.shape, inv_yhat_reshape.shape (19268, 24, 4) (19268, 24, 4)


### Performance

In [37]:
print('MAE = {}'.format(float("{:.4f}".format(mae(inv_y[:, 0], inv_yhat[:, 0])))))
print('RMSE = {}'.format(float("{:.4f}".format(sqrt(mse(inv_y[:, 0], inv_yhat[:, 0]))))))


t1 = 1
upper_threshold = 3.5
lower_threshold = 0
print("Over thresholds:", flood_threshold_t1(inv_yhat_reshape, t1, upper_threshold))
print("Under thresholds:", drought_threshold_t1(inv_yhat_reshape, t1, lower_threshold))

MAE = 0.0373
RMSE = 0.0533
S1, S25A, S25B, S26 time steps: 89, 93, 111, 133
S1, S25A, S25B, S26 areas: 12.6076, 13.8272, 17, 22.523
TOTAL time steps: 426; TOTAL areas: 65.6809
--------------------------------------------------
Over thresholds: None
S1, S25A, S25B, S26 time steps: 1354, 1359, 1238, 1417
S1, S25A, S25B, S26 areas: -390.4474, -376.9404, -339.7464, -416.1338:
TOTAL time steps: 5368; TOTAL areas: -1523.2681
--------------------------------------------------
Under thresholds: None
